In [1]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import codecs

from torch.utils.data import TensorDataset, DataLoader

%matplotlib inline

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
df = pd.read_csv(r".\rus-eng\rus.txt",sep='\t',names=['eng','rus','c'],encoding='utf-8')[['eng','rus']]

In [4]:
df

,eng,rus
0,Go.,Марш!
1,Go.,Иди.
2,Go.,Идите.
3,Hi.,Здравствуйте.
4,Hi.,Привет!
...,...,...
496054,"At a moment when our economy is growing, our b...","В тот момент, когда наша экономика растёт, наш..."
496055,"When I was younger, I hated going to weddings....","Когда я была помоложе, я ненавидела ходить на ..."
496056,Since there are usually multiple websites on a...,"Поскольку сайтов, посвящённых какой-либо теме,..."
496057,If someone who doesn't know your background sa...,"Если кто-то незнакомый говорит, что вы говорит..."


In [5]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [6]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s,lang = 'eng'):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    if lang=='eng':
        s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    elif lang=='rus':
        r = re.compile(r"[^ЁёА-я.!?]+")
        r.sub(" ", s)
    return s

In [7]:
def readLangs(df, reverse=False):
  
    if reverse:
        lang1 = df.columns[1]
        lang2 = df.columns[0]
    else:
        lang1 = df.columns[0]
        lang2 = df.columns[1]
    
    print(f"Reading lines {lang1} to {lang2}...")
    
    pairs = df.copy()
    
    pairs[lang1] = pairs[lang1].apply(lambda x: normalizeString(x,lang=lang1))
    pairs[lang2] = pairs[lang2].apply(lambda x: normalizeString(x,lang=lang2))
    
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)
    
    return input_lang, output_lang, pairs[[lang1,lang2]]

In [8]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPairs(df):
    lang1 = df.columns[0]
    lang2 = df.columns[1]
    return df[
                     (df[lang1].str.split(" ").map(len)<MAX_LENGTH) 
                   & (df[lang2].str.split(" ").map(len)<MAX_LENGTH)
                   & (df[lang2].str.startswith(eng_prefixes))
                ].reset_index(drop=True)

In [9]:
def prepareData(df, reverse=False):
    input_lang, output_lang, pairs = readLangs(df, reverse=True)
    print("Read %s sentence pairs" % pairs.shape[0])
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % pairs.shape[0])
    print("Counting words...")
    _ = pairs[pairs.columns[0]].apply(input_lang.addSentence)
    _ = pairs[pairs.columns[1]].apply(output_lang.addSentence)
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData(df,reverse=True)
print(pairs.sample(1))

Reading lines rus to eng...
Read 496059 sentence pairs
Trimmed to 28727 sentence pairs
Counting words...
Counted words:
rus 10804
eng 4306
                        rus                        eng
14675  она не моя девушка .  she s not my girlfriend .


### Тренировочный и проверочные датасеты

In [10]:
#get train and test datasets
testcoeff = 0.1
test_pairs  = pairs.sample(int(pairs.shape[0]*testcoeff))
train_pairs = pairs[~pairs.index.isin(test_pairs.index)].to_numpy()
test_pairs = test_pairs.to_numpy()
print(f'train shape: {train_pairs.shape}, test_shape: {test_pairs.shape}')

train shape: (25855, 2), test_shape: (2872, 2)


In [11]:
train_pairs

array([['мне девятнадцать лет .', 'i m .'],
       ['со мнои все в порядке .', 'i m ok .'],
       ['у меня все хорошо .', 'i m ok .'],
       ...,
       ['она улыбнулась в ответ на его нежныи взгляд .',
        'she smiled in response to his affectionate glance .'],
       ['каждую субботу после обеда она играет в теннис .',
        'she spends every saturday afternoon playing tennis .'],
       ['после аварии она перестала бывать на людях .',
        'she stopped appearing in public after her accident .']],
      dtype=object)

The Encoder
-----------

In [12]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------

In [13]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [14]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [15]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [16]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [17]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(train_pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [18]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [19]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        #print(sentence.shape)
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [20]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(test_pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [21]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

1m 29s (- 20m 56s) (5000 6%) 3.1064
2m 49s (- 18m 20s) (10000 13%) 2.6007
4m 9s (- 16m 36s) (15000 20%) 2.3511
5m 29s (- 15m 4s) (20000 26%) 2.1200
6m 46s (- 13m 33s) (25000 33%) 1.9889
8m 4s (- 12m 6s) (30000 40%) 1.8606
9m 20s (- 10m 40s) (35000 46%) 1.7633
10m 37s (- 9m 17s) (40000 53%) 1.6643
11m 53s (- 7m 55s) (45000 60%) 1.5774
13m 9s (- 6m 34s) (50000 66%) 1.5083
14m 26s (- 5m 14s) (55000 73%) 1.4422
15m 42s (- 3m 55s) (60000 80%) 1.3883
16m 59s (- 2m 36s) (65000 86%) 1.3421
18m 15s (- 1m 18s) (70000 93%) 1.2798
19m 32s (- 0m 0s) (75000 100%) 1.2624


In [22]:
evaluateRandomly(encoder1, attn_decoder1)

> я готов идти за вами .
= i m ready to follow you .
< i am ready to follow you . <EOS>

> я в этом уверен .
= i m certain of it .
< i m absolutely sure of it . <EOS>

> я сеичас иду в отель .
= i m going to the hotel now .
< i m going to the in now . <EOS>

> они пытаются помочь .
= they re trying to help .
< they re trying to help . <EOS>

> они работают наверху .
= they re working upstairs .
< they re working on . <EOS>

> я просто болтаюсь .
= i m just hanging out .
< i m just a . . <EOS>

> они сумасшедшие .
= they re crazy .
< they re crazy . <EOS>

> мы недалеко от дома тома .
= we re not far from tom s .
< we re behind of tom . <EOS>

> вы гении .
= you re a genius .
< you re a genius . <EOS>

> я об этом не думаю .
= i m not thinking about it .
< i m not going about it . <EOS>



### На основе скалярного произведения

In [113]:
class AttnDecoderRNN_DOT(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN_DOT, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
        attn_weights = F.softmax(
            torch.matmul(
                encoder_outputs,hidden[0].squeeze()* self.hidden_size**-0.5
                ).unsqueeze(0)
            , dim=1)
        
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [114]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN_DOT(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

1m 18s (- 18m 23s) (5000 6%) 3.0899
2m 31s (- 16m 27s) (10000 13%) 2.6058
3m 45s (- 15m 1s) (15000 20%) 2.3548
4m 58s (- 13m 41s) (20000 26%) 2.1562
6m 12s (- 12m 25s) (25000 33%) 2.0032
7m 27s (- 11m 10s) (30000 40%) 1.8859
8m 42s (- 9m 56s) (35000 46%) 1.7555
9m 56s (- 8m 41s) (40000 53%) 1.6533
11m 11s (- 7m 27s) (45000 60%) 1.5728
12m 26s (- 6m 13s) (50000 66%) 1.5008
13m 40s (- 4m 58s) (55000 73%) 1.4226
14m 56s (- 3m 44s) (60000 80%) 1.3417
16m 10s (- 2m 29s) (65000 86%) 1.3002
17m 25s (- 1m 14s) (70000 93%) 1.2211
18m 42s (- 0m 0s) (75000 100%) 1.1754


In [115]:
evaluateRandomly(encoder1, attn_decoder1)

> ты в этом уверена, да ?
= you re sure about this right ?
< you re not that around aren t you ? <EOS>

> он в пути .
= he s on his way .
< he is on the way . <EOS>

> она темнокожая .
= she is dark skinned .
< she is young . <EOS>

> мы избегаем тома .
= we re avoiding tom .
< we re going to wait for tom . <EOS>

> я очень рассеянная .
= i m very absent minded .
< i m very grateful . <EOS>

> рад за вас обоих .
= i m happy for both of you .
< i m glad to see you . <EOS>

> он очень робкии .
= he s very timid .
< he s very shy . <EOS>

> простите, что утром опоздал .
= i m sorry i was late this morning .
< i m sorry i m late . <EOS>

> не уверен, что это случится .
= i m not sure that that ll happen .
< i m not sure this is is . . <EOS>

> я выигрываю .
= i m winning .
< i m a . . <EOS>



### На основе MPL

In [277]:
class AttnDecoderRNN_MLP(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN_MLP, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        
        #self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.mlp =  nn.Sequential(
                    nn.Linear(self.hidden_size*2, 512),
                    nn.ReLU(),
                    nn.Dropout(0.5),
                    nn.Linear(512, 512),
                    nn.ReLU(),
                    nn.Dropout(0.5),
                    nn.Linear(512, self.max_length)
                    )
        
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)
        
    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)
       
        attn_weights = F.softmax(
            self.mlp(
                torch.cat(
                        (encoder_outputs,hidden[0].repeat(self.max_length,1)), dim=1
                    )
                ).mean(dim=1).unsqueeze(0),
        #     self.mlp((encoder_outputs+hidden[0]).flatten().unsqueeze(0)),
        #    #self.mlp(torch.cat((encoder_outputs,hidden[0].repeat(self.max_length,1)), dim=1).flatten(0).unsqueeze(0)).mean(dim=1).unsqueeze(0),
        #    #self.mlp(encoder_outputs).mean(dim=1).unsqueeze(0),
        #    #self.mlp(encoder_outputs.flatten(0).unsqueeze(0))
            dim=1
            )
        #attn_weights = self.mlp((encoder_outputs*hidden[0]).flatten().unsqueeze(0)),

        
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))
        
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [278]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN_MLP(hidden_size, output_lang.n_words, dropout_p=0.1).to(device)

trainIters(encoder1, attn_decoder1, 75000, print_every=5000)

1m 40s (- 23m 22s) (5000 6%) 3.0996
3m 15s (- 21m 9s) (10000 13%) 2.6038
4m 51s (- 19m 26s) (15000 20%) 2.3760
6m 27s (- 17m 46s) (20000 26%) 2.1552
8m 4s (- 16m 8s) (25000 33%) 1.9992
9m 42s (- 14m 33s) (30000 40%) 1.8598
11m 20s (- 12m 57s) (35000 46%) 1.7277
12m 58s (- 11m 21s) (40000 53%) 1.6295
14m 36s (- 9m 44s) (45000 60%) 1.5684
16m 13s (- 8m 6s) (50000 66%) 1.4648
17m 49s (- 6m 29s) (55000 73%) 1.4120
19m 24s (- 4m 51s) (60000 80%) 1.3247
21m 0s (- 3m 13s) (65000 86%) 1.2607
22m 36s (- 1m 36s) (70000 93%) 1.2144
24m 13s (- 0m 0s) (75000 100%) 1.1692


In [280]:
evaluateRandomly(encoder1, attn_decoder1)

> у меня все еще немного кружится голова .
= i m still a little woozy .
< i m still a little sick . <EOS>

> я просто счастлив, что все так хорошо закончилось .
= i m just happy it ended so well .
< i m just doing that that s is . <EOS>

> я счастлив тратить деньги на книги .
= i am happy to spend money on books .
< i m three inches my life in . . <EOS>

> вы еще молодая .
= you re still young .
< you re still young . <EOS>

> он так же силен, как всегда .
= he is as strong as ever .
< he s as smart as ever . <EOS>

> я собираюсь покинуть страну .
= i m going to leave the country .
< i m going to leave my . . <EOS>

> я все еще в австралии .
= i m still in australia .
< i m still in australia . <EOS>

> ты права .
= you re right .
< you re a . <EOS>

> я подвыпившая .
= i m tipsy .
< i m a . <EOS>

> я смущен .
= i m confused .
< i m embarrassed . <EOS>



Применение Scoring Function на основе скалярного произведения и с использованием MLP, немного уменьшило ошибку, 
однако совсем не драматично. На тестовых данных, вроде как работает получше.